In [11]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [13]:
%autoreload
from pathlib import Path
import sys
root = Path().resolve().absolute().parent.parent
print(root)
sys.path.append(str(root))

from src.pipeline import Pipeline, BuilingIdsEnum
pipe = Pipeline()



/Users/matsalexander/Desktop/SolarEnergyImpact


In [14]:
building_b = pipe.get_data(BuilingIdsEnum.B)
building_c = pipe.get_data(BuilingIdsEnum.C)
building_a = pipe.get_data(BuilingIdsEnum.A)

# from november 1. to 1. mars only use data from dataset b 
# else use data from dataset c and a

# remove from 2024-01-15 from dataset A
mask_b = (building_b['timestamp'] >= '2024-01-15') & (building_b['timestamp'] <= '2024-01-15')
building_b = building_b[~mask_b]

start_winter = '2023-11-01'
end_winter = '2024-03-01'
mask_b = (building_b['timestamp'] >= start_winter) & (building_b['timestamp'] <= end_winter)
building_b = building_b[mask_b]

mask_c = (building_c['timestamp'] < start_winter) | (building_c['timestamp'] > end_winter)
building_c = building_c[mask_c]

# building a, only before 2023-09-17.
end_dataset_a = '2023-09-17'
mask_a = (building_a['timestamp'] < end_dataset_a)
building_a = building_a[mask_a]



# normalize the value_import by area and call it value for main and building_b
building_b['value'] = building_b['value_import'] / building_b['area']
building_c['value'] = building_c['net_consumption'] / building_c['area']
building_a['value'] = building_a['value_import'] / building_a['area']


combined_df = pd.concat([building_a, building_b, building_c], ignore_index=True)
# reset index
combined_df = combined_df.reset_index(drop=True)
# Perform the train-test split with stratification based on 'building_id'
train_data, test_data = train_test_split(
    combined_df,
    test_size=0.20,
    random_state=42,
    stratify=combined_df['building']
)

# select features
target = "value"
features = ["timestamp", "temperature"] #, "cloud_fraction", "precipitation", "area", "wind_speed"

train_data = train_data[features + [target]]
test_data = test_data[features + [target]]



In [ ]:
# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN']
    )

No path specified. Models will be saved in: "AutogluonModels/ag-20241113_193908"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       13.50 GB / 32.00 GB (42.2%)
Disk Space Avail:   607.65 GB / 926.35 GB (65.6%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal 

[1000]	valid_set's l1: 0.00495001
[2000]	valid_set's l1: 0.00467688
[3000]	valid_set's l1: 0.00451896
[4000]	valid_set's l1: 0.00441288
[5000]	valid_set's l1: 0.00433018
[6000]	valid_set's l1: 0.00426813
[7000]	valid_set's l1: 0.00421587
[8000]	valid_set's l1: 0.00418395
[9000]	valid_set's l1: 0.00415196
[10000]	valid_set's l1: 0.00413354
[1000]	valid_set's l1: 0.00482558
[2000]	valid_set's l1: 0.00456458
[3000]	valid_set's l1: 0.00442977
[4000]	valid_set's l1: 0.00435085
[5000]	valid_set's l1: 0.0043035
[6000]	valid_set's l1: 0.00426043
[7000]	valid_set's l1: 0.00423126
[8000]	valid_set's l1: 0.0042121
[9000]	valid_set's l1: 0.00419536
[10000]	valid_set's l1: 0.00418205
[1000]	valid_set's l1: 0.00495874
[2000]	valid_set's l1: 0.00471132
[3000]	valid_set's l1: 0.00457823
[4000]	valid_set's l1: 0.00448806


	Ran out of time, early stopping on iteration 4903. Best iteration is:
	[4903]	valid_set's l1: 0.0044302


[1000]	valid_set's l1: 0.00480689
[2000]	valid_set's l1: 0.00456495
[3000]	valid_set's l1: 0.00442552
[4000]	valid_set's l1: 0.00433315


	Ran out of time, early stopping on iteration 4091. Best iteration is:
	[4073]	valid_set's l1: 0.00432586


[1000]	valid_set's l1: 0.00489035
[2000]	valid_set's l1: 0.00462095
[3000]	valid_set's l1: 0.0044993
[4000]	valid_set's l1: 0.0044304


	Ran out of time, early stopping on iteration 4256. Best iteration is:
	[4256]	valid_set's l1: 0.00441864


[1000]	valid_set's l1: 0.00528375
[2000]	valid_set's l1: 0.00499781
[3000]	valid_set's l1: 0.00485029


In [ ]:
# AutogluonModels/ag-
# Evaluate on test data
performance = predictor.evaluate(test_data)
# best model: ag-20241022_161331

print("Evaluation Performance:")
performance
# reset index 
test_data = test_data.reset_index(drop=True)
# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions


Computing feature importance via permutation shuffling for 3 features using 2074 rows with 5 shuffle sets...


Evaluation Performance:


	160.77s	= Expected runtime (32.15s per shuffle set)
	115.89s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
             importance    stddev       p_value  n  p99_high   p99_low
temperature    0.005131  0.000166  1.316034e-07  5  0.005473  0.004789
timestamp      0.005019  0.000123  4.318946e-08  5  0.005272  0.004766
wind_speed     0.000660  0.000010  5.865023e-09  5  0.000680  0.000640


In [ ]:
performance

{'mean_absolute_error': -0.0024578363248878984,
 'root_mean_squared_error': -0.003673289387084216,
 'mean_squared_error': -1.3493054921265535e-05,
 'r2': 0.8163439905819311,
 'pearsonr': 0.9037184977614303,
 'median_absolute_error': -0.0014704587917796462}

In [ ]:
# model location => AutogluonModels/ag-20241016_095906
main_building = pipe.get_data(BuilingIdsEnum.MAIN)

data_predict = main_building[features]
data_predict


,timestamp,temperature,wind_speed
0,2023-07-01 00:00:00,13.6,1.6
1,2023-07-01 01:00:00,13.2,2.0
2,2023-07-01 02:00:00,12.3,1.6
3,2023-07-01 03:00:00,11.9,0.6
4,2023-07-01 04:00:00,11.9,0.2
...,...,...,...
10338,2024-09-03 18:00:00,17.8,2.5
10339,2024-09-03 19:00:00,17.8,1.8
10340,2024-09-03 20:00:00,17.7,1.1
10341,2024-09-03 21:00:00,18.0,3.4


In [ ]:
prediciton1 = predictor.predict(data_predict)

In [ ]:
# save predicitons as a csv in data folder from root.
import datetime
from pathlib import Path


prediciton1_df = pd.DataFrame(prediciton1)
date_time = datetime.datetime.now().strftime("%Y%m%d_%H")
my_path = Path().resolve().parent.parent / 'data'/ "pred" / 'prediction_winter.csv'
if my_path.exists():
    my_path = Path().resolve().parent.parent / 'data'/ "pred" / f'prediction_winter_{date_time}.csv'
# create folder
my_path.parent.mkdir(parents=True, exist_ok=True)
prediciton1_df.to_csv(my_path, index=False)